# Download do repositório: Clojure

Primeiro devemos realizar o download do programa Clojure usando o repositório defects4J instalado no servidor Lucy.
O comando abaixo efetua o download da ferramenta na pasta: **adilsonTeste**

``` bash
/home/sin5022-adm/defects4j/framework/bin/defects4j checkout -p Closure -v 18f -w /home/evosuite/adilsonTeste/Clojure_18_fixed
```

Para realizar o download da versão com bug usamos o mesmo comando com o parâmetro v, como abaixo:

``` bash
/home/sin5022-adm/defects4j/framework/bin/defects4j checkout -p Closure -v 18b -w /home/evosuite/adilsonTeste/Clojure_18_bug
```

# Geração de testes automatizada

Vale ressaltar que o parâmetro -A força a execucao sobre todas as classes de interesse (de acordo com a doc) o tempo de execução de cada caso foi aproximadamente de 4h após adicionar o parâmetro -A.

## Criteria: branch

``` bash
nohup /home/sin5022-adm/defects4j/framework/bin/run_evosuite.pl -p Closure -v 18b -c branch -A -o /home/evosuite/adilsonTeste/outputTeste/ -n 1 -t /home/evosuite/adilsonTeste/Clojure_18_bug/ &
```

## Criteria: weakmutation

``` bash
nohup /home/sin5022-adm/defects4j/framework/bin/run_evosuite.pl -p Closure -v 18b -c weakmutation -A -o /home/evosuite/adilsonTeste/outputTeste/ -n 2 -t /home/evosuite/adilsonTeste/Clojure_18_bug/ &
```

## Criteria: strongmutation

``` bash
nohup /home/sin5022-adm/defects4j/framework/bin/run_evosuite.pl -p Closure -v 18b -c strongmutation -A -o /home/evosuite/adilsonTeste/outputTeste/ -n 3 -t /home/evosuite/adilsonTeste/Clojure_18_bug/ &
```

Vale ressaltar que o comando acima gera os testes, executa os testes sobre a classe defeituosa, calcula a cobertura entre outras informações que serão relatadas abaixo. Cada execução gerou um arquivo de log: i) Closure.18b.branch.1.log (1 Mb); ii) Closure.18b.weakmutation.2.log (1 Mb) ; iii) Closure.18b.strongmutation.3.log ( 120 Mb ocorreu muita exception, o stacktrace foi o responsável por esse tamanho todo) onde são armazenadas as informações sobre a execução.

Foi construído um parser para extrair a informação desses logs, como não teho acesso root no servidor (para instalar pacotes python) executei o parser em minha máquina local, segue código do parser:

In [4]:
import re
import sys
import pandas as pd
import numpy as np


def parse_evosuite_log_file(path):
     
     df_log_evo = pd.DataFrame(columns=('class_name',
                                        'number_of_goals',
                                        'number_of_tests',
                                        'length_of_testes',
                                        'ga_zero_fitness',
                                        'ga_max_time',
                                        'ga_shutdown_test_writer',
                                        'test_suit_mutation_score',
                                        'time_spent_to_execute_tests',
                                        'test_suit_coverage',
                                        'num_coverage_goals',
                                        'criterion',
                                        'coverage_of_criterion',
                                        'search_finished_after',
                                        'num_generations',
                                        'num_statements',   
                                        'best_ind_fitness',
                                        'permissions',
                                        'warnings',
                                        'exceptions'
                                        ))
     class_name = None
     number_of_goals = None
     number_of_tests = None
     length_of_testes = None
     ga_zero_fitness = None
     ga_max_time = None
     ga_shutdown_test_writer = None
     test_suit_mutation_score = None
     time_spent_to_execute_tests = None
     test_suit_coverage = None
     num_coverage_goals = None
     criterion = None
     coverage_of_criterion = None
     search_finished_after = None
     num_generations = None
     num_statements    = None
     best_ind_fitness = None
     permissions = '' 
     warnings = ''
     exceptions = ''
     
     
     marcacaoPermissao = False  
     marcacaoExceptions = False  
     
     with open(path) as fp:  
         
       line = fp.readline()
       cnt = 1
       while line:
           
           #warnings
           if(line.find("WARN") != -1):
               warnings = warnings + line.split("WARN")[1].strip() + "\n //"

           #exceptions
           if( marcacaoExceptions == True or line.find( "ERROR") != -1 ):
               marcacaoExceptions = True
               
               #Acabou a descricao de erro
               if( str(line[0]).find('*') == 0 and line.find( "ERROR") == -1 ):
                   marcacaoExceptions = False
                   
               if(str(line[0]).find('*') != 0):
                   exceptions = exceptions + line + " // "
                   
               
           if( marcacaoPermissao == True or line.find( "Permissions denied during test execution") != -1 ):
               marcacaoPermissao = True
               
               #Acabou a descricao de erro
               if( str(line[0]).find('*') == 0 and line.find( "Permissions denied during test execution") == -1 ):
                   marcacaoPermissao = False
                   
               if(str(line[0]).find('*') != 0):
                   permissions = permissions + line + " // "
           
           if(line.find("Going to generate test cases for class:") != -1):
               l = line.strip().split("class:")[1]
               class_name = l.strip()
           
           if(line.find("Total number of goals") != -1):
               l = line.strip().split("goals:")
               number_of_goals = l[1].split(":")[0].strip()

           if(line.find("tests with total length") != -1):
               l = line.strip().split(" tests with total length ")
               number_of_tests = l[0].replace('* Generated ','').split(":")[0].strip()               
               length_of_testes = l[1].strip()
               
           #GA-Budget
           if(line.find("ZeroFitness") != -1):
               l = line.strip().split(":")
               ga_zero_fitness = l[1].strip()
               
           if(line.find("MaxTime") != -1):
               l = line.strip().split(":")
               ga_max_time = l[1].strip()
           
           if(line.find("ShutdownTestWriter") != -1):
               l = line.strip().split(":")
               ga_shutdown_test_writer = l[1].strip()
           
           if(line.find("Resulting test suite's mutation score") != -1):
               l = line.strip().split("score:")
               test_suit_mutation_score = l[1].strip()

           if(line.find("Time spent executing tests") != -1):
               l = line.strip().split("tests:")
               time_spent_to_execute_tests = l[1].strip()
           
           if(line.find("Resulting test suite's coverage") != -1):
               l = line.strip().split("coverage:")
               test_suit_coverage = l[1].strip()
            
           if(line.find("Number of covered goals") != -1):
               l = line.strip().split("goals:")
               num_coverage_goals = l[1].split(":")[0].strip()
               
           if(line.find("Coverage of criterion") != -1):
               l = line.strip().split("criterion")
               criterion = l[1].split(":")[0].strip()
               coverage_of_criterion = l[1].split(":")[1].strip()
                
           if(line.find("Search finished after") != -1):              
               #Quebra em 3 blocos de informacao
               l = line.strip().split(", ")

               #tempo de busca
               l0tmp = l[0].split(" and ")

               #tempo
               search_finished_after = l0tmp[0].split("after ")[1].strip()

               #geracoes
               num_generations = l0tmp[1].split(" generations")[0].strip()
               
               #statements
               l1tmp = l[1].split(" ")
               num_statements = l1tmp[0].strip()
               
               #individual fitness
               l2tmp = l[2].split(": ")
               best_ind_fitness = l2tmp[1].strip()
               
               
           #Linha de erros de permissão

           if(line.find("Done") != -1):
               #add to dataframe
               df_log_evo = df_log_evo.append({
                                                'class_name':class_name,
                                                'number_of_goals':number_of_goals,
                                                'number_of_tests':number_of_tests,
                                                'length_of_testes':length_of_testes,
                                                'ga_zero_fitness':ga_zero_fitness,
                                                'ga_max_time':ga_max_time,
                                                'ga_shutdown_test_writer':ga_shutdown_test_writer,
                                                'test_suit_mutation_score':test_suit_mutation_score,
                                                'time_spent_to_execute_tests':time_spent_to_execute_tests,
                                                'test_suit_coverage':test_suit_coverage,
                                                'num_coverage_goals':num_coverage_goals,
                                                'criterion':criterion,
                                                'coverage_of_criterion':coverage_of_criterion,
                                                'search_finished_after':search_finished_after,
                                                'num_generations':num_generations,
                                                'num_statements':num_statements,   
                                                'best_ind_fitness':best_ind_fitness,
                                                'permissions':permissions,
                                                'warnings':warnings,
                                                'exceptions':exceptions
                                           }, ignore_index=True)
                
               class_name = None
               number_of_goals = None
               number_of_tests = None
               length_of_testes = None
               ga_zero_fitness = None
               ga_max_time = None
               ga_shutdown_test_writer = None
               test_suit_mutation_score = None
               time_spent_to_execute_tests = None
               test_suit_coverage = None
               num_coverage_goals = None
               criterion = None
               coverage_of_criterion = None
               search_finished_after = None
               num_generations = None
               num_statements    = None
               best_ind_fitness = None
               permissions = ''
               warnings = ''
               exceptions = ''
            
           line = fp.readline()
           cnt += 1
    
     return(df_log_evo)
pass

#gerar o csv compactado dos logs.
##Branch
#path = "Closure.18b.branch.1.log"
#config_evo_run = parse_evosuite_log_file(path)
#config_evo_run.to_csv('parseado_branch.csv',sep = ';')


#weakmutation
#path = "Closure.18b.weakmutation.2.log"
#config_evo_run = parse_evosuite_log_file(path)
#config_evo_run.to_csv('parseado_weakmutation.csv',sep = ';')


#strongmutation #apontar para o desktop eh muit grande coisa de 116Mb
#path = "Closure.18b.strongmutation.3.log"
#config_evo_run = parse_evosuite_log_file(path)
#config_evo_run.to_csv('parseado_strongmutation_no_error.csv',sep = ';')

O output dessas 3 chamadas são 3 arquivos csv com os dados tabulados.

# Análise de Resultados da geração e execução dos testes

Abaixo podemos ver o código para sumarizar os csvs permitindo um overview das execuções:

In [5]:
import re
import sys
import pandas as pd
import numpy as np


#Conventions for the pd command line
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)


def analisa_csv(path_csv):
    
    print("=====================================================")
    print("=====================================================")
    print("=====================================================")
    
    branchcsv = pd.read_csv(path_csv, sep = ';')
    
    #remove o index
    branchcsv = branchcsv.drop(['Unnamed: 0'],axis = 1)
    
    print("Número de classes criadas teste: {}".format(branchcsv.shape[0]))
    print("Número de atributos das classes criadas teste: {}".format(branchcsv.shape[1]))
    print("Total de casos de teste criados para todas as classes: {}".format(branchcsv.number_of_tests.sum()))
    print("Média de testes criados por classe: {}".format( round(branchcsv.number_of_tests.sum()/branchcsv.shape[0], 2) ))
    print("Tamanho médio(num statements) dos casos de testes criados: {}".format( round(branchcsv.length_of_testes.mean(), 2) ))
    print("Número de classes cujos testes geraram warning: {}".format( round(branchcsv.warnings.count(), 2) ))
    print("Número de classes cujos testes geraram exceptions: {}".format( round(branchcsv.exceptions.count(), 2) ))
    print("Número de classes cujos testes tiveram problemas de permissão: {}".format( round(branchcsv.permissions.count(), 2) ))
    
    print("=======================================================================")
    
    notNatest_suit_mutation_score = branchcsv[branchcsv.test_suit_mutation_score.notnull()].copy(deep = True)
    Natest_suit_mutation_score = branchcsv[branchcsv.test_suit_mutation_score.isna()].copy(deep = True)
    notNatest_suit_mutation_score['test_suit_mutation_score_noPercent'] = notNatest_suit_mutation_score.test_suit_mutation_score.str.replace('%','')
    
    notNatest_suit_mutation_score.test_suit_mutation_score_noPercent = notNatest_suit_mutation_score.test_suit_mutation_score_noPercent.astype(int)
    print("max mutation score: {}".format(notNatest_suit_mutation_score.test_suit_mutation_score_noPercent.max()))
    print("min mutation score: {}".format(notNatest_suit_mutation_score.test_suit_mutation_score_noPercent.min()))
    print("mean mutation score: {}".format(round(notNatest_suit_mutation_score.test_suit_mutation_score_noPercent.mean(),2)))
    print("number of NaN: {}".format(Natest_suit_mutation_score.test_suit_mutation_score.isna().sum()))
    
    print("=======================================================================")
    
    notNatime_spent_to_execute_tests = branchcsv[branchcsv.time_spent_to_execute_tests.notnull()].copy(deep = True)
    Natime_spent_to_execute_tests = branchcsv[branchcsv.time_spent_to_execute_tests.isna()].copy(deep = True)
    notNatime_spent_to_execute_tests['time_spent_to_execute_tests_no_unity'] = notNatime_spent_to_execute_tests['time_spent_to_execute_tests'].str.replace('ms','')
    notNatime_spent_to_execute_tests.time_spent_to_execute_tests_no_unity = notNatime_spent_to_execute_tests.time_spent_to_execute_tests_no_unity.astype(int)
    
    print("max time_spent_to_execute_tests: {} (seg)".format(notNatime_spent_to_execute_tests.time_spent_to_execute_tests_no_unity.max()/1000))
    print("min time_spent_to_execute_tests: {} (seg)".format(notNatime_spent_to_execute_tests.time_spent_to_execute_tests_no_unity.min()/1000))
    print("mean time_spent_to_execute_tests: {} (seg)".format(round(notNatime_spent_to_execute_tests.time_spent_to_execute_tests_no_unity.mean()/1000),2))
    print("number of NaN: {}".format(Natime_spent_to_execute_tests.time_spent_to_execute_tests.isna().sum()))
        
    #Limpeza
    print("=====================================================")
    notNatest_suit_coverage = branchcsv[branchcsv.test_suit_coverage.notnull()].copy(deep = True)
    Natest_suit_coverage = branchcsv[branchcsv.test_suit_coverage.isna()].copy(deep = True)
    notNatest_suit_coverage['test_suit_coverage_no_unity'] = notNatest_suit_coverage['test_suit_coverage'].str.replace('%','')
    notNatest_suit_coverage.test_suit_coverage_no_unity = notNatest_suit_coverage.test_suit_coverage_no_unity.astype(int)
    
    print("max test_suit_coverage: {} (%)".format(notNatest_suit_coverage.test_suit_coverage_no_unity.max()))
    print("min test_suit_coverage: {} (%)".format(notNatest_suit_coverage.test_suit_coverage_no_unity.min()))
    print("mean test_suit_coverage: {} (%)".format(round(notNatest_suit_coverage.test_suit_coverage_no_unity.mean(),2)))
    print("number of NaN: {}".format(Natest_suit_coverage.test_suit_coverage.isna().sum()))
    print("=====================================================")
    
    #Limpeza
    notNacoverage_of_criterion = branchcsv[branchcsv.coverage_of_criterion.notnull()].copy(deep = True)
    Nacoverage_of_criterion = branchcsv[branchcsv.coverage_of_criterion.isna()].copy(deep = True)

    notNacoverage_of_criterion['coverage_of_criterion_no_unity'] = notNacoverage_of_criterion['coverage_of_criterion'].str.replace('%','')
    notNacoverage_of_criterion['coverage_of_criterion_no_unity'] = notNacoverage_of_criterion['coverage_of_criterion_no_unity'].str.replace(' \(no goals\)','')
    notNacoverage_of_criterion.coverage_of_criterion_no_unity = notNacoverage_of_criterion.coverage_of_criterion_no_unity.astype(int)
    
    
    print("max coverage_of_criterion_no_unity: {} (%)".format(notNacoverage_of_criterion.coverage_of_criterion_no_unity.max()))
    print("min coverage_of_criterion_no_unity: {} (%)".format(notNacoverage_of_criterion.coverage_of_criterion_no_unity.min()))
    print("mean coverage_of_criterion_no_unity: {} (%)".format(round(notNacoverage_of_criterion.coverage_of_criterion_no_unity.mean(),2)))
    print("number of NaN: {}".format(Nacoverage_of_criterion.coverage_of_criterion.isna().sum()))
    
    print("=====================================================")
    
    #
    print("GA - métricas:")
    notNasearch_finished_after = branchcsv[branchcsv.search_finished_after.notnull()].copy(deep = True)
    Nasearch_finished_after = branchcsv[branchcsv.search_finished_after.isna()].copy(deep = True)
    
    notNasearch_finished_after['search_finished_after_no_unity'] = notNasearch_finished_after['search_finished_after'].str.replace('s','')
    notNasearch_finished_after.search_finished_after_no_unity = notNasearch_finished_after.search_finished_after_no_unity.astype(int)

    print("=====================================================")
    print("max search_finished_after_no_unity: {} (s)".format(notNasearch_finished_after.search_finished_after_no_unity.max()))
    print("min search_finished_after_no_unity: {} (s)".format(notNasearch_finished_after.search_finished_after_no_unity.min()))
    print("mean search_finished_after_no_unity: {} (s)".format(round(notNasearch_finished_after.search_finished_after_no_unity.mean(),2)))
    print("number of NaN: {}".format(Nasearch_finished_after.search_finished_after.isna().sum()))
    
    print("=====================================================")
    notNanum_generations = branchcsv[branchcsv.num_generations.notnull()].copy(deep = True)
    Nanum_generations = branchcsv[branchcsv.num_generations.isna()].copy(deep = True)


    print("max num_generations: {}".format(notNanum_generations.num_generations.max()))
    print("min num_generations: {}".format(notNanum_generations.num_generations.min()))
    print("mean num_generations: {}".format(round(notNanum_generations.num_generations.mean(),2)))
    print("number of NaN: {}".format(Nanum_generations.num_generations.isna().sum()))
    
    print("=====================================================")
    notNanum_statements = branchcsv[branchcsv.num_statements.notnull()].copy(deep = True)
    Nanum_statements = branchcsv[branchcsv.num_statements.isna()].copy(deep = True)
    
    print("max num_statements: {}".format(notNanum_statements.num_statements.max()))
    print("min num_statements: {}".format(notNanum_statements.num_statements.min()))
    print("mean num_statements: {}".format(round(notNanum_statements.num_statements.mean(),2)))
    print("number of NaN: {}".format(Nanum_statements.num_statements.isna().sum()))
    
    print("=====================================================")
    notNabest_ind_fitness = branchcsv[branchcsv.best_ind_fitness.notnull()].copy(deep = True)
    Nabest_ind_fitness = branchcsv[branchcsv.best_ind_fitness.isna()].copy(deep = True)
    
    print("max best_ind_fitness: {}".format(round(notNabest_ind_fitness.best_ind_fitness.max(),2)))
    print("min best_ind_fitness: {}".format(notNabest_ind_fitness.best_ind_fitness.min()))
    print("mean best_ind_fitness: {}".format(round(notNabest_ind_fitness.best_ind_fitness.mean(),2)))
    print("number of NaN: {}".format(Nabest_ind_fitness.best_ind_fitness.isna().sum()))
    
    print("=====================================================")
    print("=====================================================")
    print("=====================================================")
    print("Tempo Geral")
    
    #Tempo total rodando testes: converter para horas
    print("sum time_spent_to_execute_tests: {} (horas)".format(round( notNatime_spent_to_execute_tests.time_spent_to_execute_tests_no_unity.sum()/(1000*60*60),2 ) ) ) 
    
    #Tempo total de pesquisa do GA:
    print("sum search_finished_after_no_unity: {} (horas)".format(round( notNasearch_finished_after.search_finished_after_no_unity.sum()/(60*60),2)))

    print("=====================================================")
    print("=====================================================")
    print("=====================================================")

pass

branchcsv = 'parseado_branch.csv'
weakcsv = 'parseado_weakmutation.csv'
strongcsv = 'parseado_strongmutation_no_error.csv'


## Resultados: branch

In [6]:
analisa_csv(branchcsv)

Número de classes criadas teste: 159
Número de atributos das classes criadas teste: 20
Total de casos de teste criados para todas as classes: 3260
Média de testes criados por classe: 20.5
Tamanho médio(num statements) dos casos de testes criados: 76.21
Número de classes cujos testes geraram warning: 144
Número de classes cujos testes geraram exceptions: 6
Número de classes cujos testes tiveram problemas de permissão: 60
max mutation score: 100
min mutation score: 0
mean mutation score: 54.37
number of NaN: 0
max time_spent_to_execute_tests: 112.905 (seg)
min time_spent_to_execute_tests: 0.0 (seg)
mean time_spent_to_execute_tests: 33.0 (seg)
number of NaN: 0
max test_suit_coverage: 100 (%)
min test_suit_coverage: 0 (%)
mean test_suit_coverage: 71.14 (%)
number of NaN: 0
max coverage_of_criterion_no_unity: 100 (%)
min coverage_of_criterion_no_unity: 0 (%)
mean coverage_of_criterion_no_unity: 70.5 (%)
number of NaN: 0
GA - métricas:
max search_finished_after_no_unity: 124 (s)
min search_f

## Resultados: weakcsv

In [7]:
analisa_csv(weakcsv)

Número de classes criadas teste: 160
Número de atributos das classes criadas teste: 20
Total de casos de teste criados para todas as classes: 2749.0
Média de testes criados por classe: 17.18
Tamanho médio(num statements) dos casos de testes criados: 131.52
Número de classes cujos testes geraram warning: 144
Número de classes cujos testes geraram exceptions: 9
Número de classes cujos testes tiveram problemas de permissão: 55
max mutation score: 100
min mutation score: 0
mean mutation score: 54.35
number of NaN: 1
max time_spent_to_execute_tests: 350.642 (seg)
min time_spent_to_execute_tests: 0.0 (seg)
mean time_spent_to_execute_tests: 29.0 (seg)
number of NaN: 1
max test_suit_coverage: 100 (%)
min test_suit_coverage: 0 (%)
mean test_suit_coverage: 68.53 (%)
number of NaN: 1
max coverage_of_criterion_no_unity: 100 (%)
min coverage_of_criterion_no_unity: 0 (%)
mean coverage_of_criterion_no_unity: 67.77 (%)
number of NaN: 1
GA - métricas:
max search_finished_after_no_unity: 168 (s)
min sea

## Resultados: strongcsv

In [8]:
analisa_csv(strongcsv)

Número de classes criadas teste: 149
Número de atributos das classes criadas teste: 20
Total de casos de teste criados para todas as classes: 2859.0
Média de testes criados por classe: 19.19
Tamanho médio(num statements) dos casos de testes criados: 226.02
Número de classes cujos testes geraram warning: 138
Número de classes cujos testes geraram exceptions: 0
Número de classes cujos testes tiveram problemas de permissão: 61
max mutation score: 100
min mutation score: 0
mean mutation score: 55.37
number of NaN: 2
max time_spent_to_execute_tests: 495.319 (seg)
min time_spent_to_execute_tests: 0.0 (seg)
mean time_spent_to_execute_tests: 116.0 (seg)
number of NaN: 1
max test_suit_coverage: 100 (%)
min test_suit_coverage: 0 (%)
mean test_suit_coverage: 46.72 (%)
number of NaN: 1
max coverage_of_criterion_no_unity: 100 (%)
min coverage_of_criterion_no_unity: 0 (%)
mean coverage_of_criterion_no_unity: 46.07 (%)
number of NaN: 1
GA - métricas:
max search_finished_after_no_unity: 589 (s)
min se

Nesse ponto vale a pena mostrar os 3 csvs completos para o chain.

# Next steps

# Problemas

Tentamos usar os scripts do defects4J para executar os casos de teste (d4j_test.pl e defects4j test) gerados mas a ferramenta apresentou alguns problemas que serão relatados.

### d4j_test.pl

Ao tentar executar o script: **d4j_test.pl** com o comando abaixo:

``` bash
/home/sin5022-adm/defects4j/framework/bin/d4j/d4j_test.pl  -w /home/evosuite/adilsonTeste/Clojure_18_bug -s outputTeste/Closure/evosuite-branch/1/Closure-18b-evosuite-branch.1.tar.bz2
```

Obtivemos o seguinte erro:

![alt text](pic/d4j_testpl.png)

Indicando a falta de algumas bibliotecas *Constant module*

### defects4j test

Ao tentar executar o script: **defects4j test** com o comando abaixo:

``` bash
/home/sin5022-adm/defects4j/framework/bin/defects4j test -w /home/evosuite/adilsonTeste/Clojure_18_bug -s /home/evosuite/adilsonTeste/outputTeste/Closure/evosuite-branch/1/Closure-18b-evosuite-branch.1.tar.bz2
```

Obtivemos o seguinte erro:

![alt text](pic/defects4j_test.png)

Indicando que não consegue extrair o casos de teste. Os quais estão no formato solicitados pela doc ([doc_defects4J](https://people.cs.umass.edu/~rjust/defects4j/html_doc/d4j/d4j-test.html)) da ferramenta: **Closure-18b-evosuite-branch.1.tar.bz2**. De acordo com essa doc o formato é: *project_id-version_id-test_suite_src.test_id.tar.bz2*

Seguem alguns exemplos:

1. Lang-11f-randoop.1.tar.bz2
2. Lang-12b-evosuite-weakmutation.1.tar.bz2
3. Lang-12f-evosuite-branch.1.tar.bz2



# Chamando Junity na mão

Coloquei os jars do evosuite no classpath do compilador **javac** com o comando:

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar"
```

Em seguida adicionamos o caminho absoluto das classes **java** que devem ser compiladas:

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:" testeJunity/main/java/outropackage/inside1/mult.java testeJunity/main/java/outropackage/ddd.java testeJunity/main/java/projetoCobertura/projetoCobertura/FmtRewrap.java testeJunity/test/java/projetoCobertura/projetoCobertura/FmtRewrapTeste.java
```

Compilamos, em seguida vamos executar o Junity para a classe de teste **projetoCobertura.projetoCobertura.FmtRewrapTeste**:

``` bash
java -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./testeJunity/test/java/:./testeJunity/main/java/" org.junit.runner.JUnitCore projetoCobertura.projetoCobertura.FmtRewrapTeste
```

Vale ressaltar os seguintes pontos:

1. O nome da classe de teste deve contar apenas o path relativo a classe para o projeto e não o caminho absoluto do arquivo no filesystem.

2. No classpath adicionamos o Junity, a raiz do projeto onde há o teste, a raiz do projeto onde está a classe de teste (ambos em relação ao filesystem).

Segue o output do Junity:

![alt text](pic/outputJunity.png)

# Observações: execução branch

Vale ressaltar um bug no evosuite que não permite a execução de algumas classes ao invocar o Junity $4$ por linha de comando, no github há uma discussão interessante sobre esse erro: [Github Issue](https://github.com/EvoSuite/evosuite/issues/41), resumidamente: Não há suporte para chamar por linha de comando antes do Junit 5

![alt text](pic/erroJunity4_evo.png)

## Rodando (MAC OS local) os testes unitários gerados no Lucy para a versão bug

Uma classe não compila: **Graph_ESTest.java** segue abaixo o erro de execução encontrado:

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-branch/1/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-branch/1/ -name "*.java")
```

![alt text](pic/erroComp.png)

Removendo a classe com erro:

``` bash
--Removi a classe com erro
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-branch/1/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-branch/1/ -name "*.java" -and -not -name "Graph_ESTest.java")
``` 

foi possível compilar perfeitamente ($38$ warnings sobre uso de lib deprecated). Em seguida, executamos as classes de teste geradas pelo evosuite!

``` bash
java -cp .:"hamcrest-core-1.3.jar:junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-branch/1/" org.junit.runner.JUnitCore com.google.javascript.jscomp.DisambiguateProperties_ESTest com.google.javascript.jscomp.CodingConvention_ESTest com.google.javascript.jscomp.ConstCheck_ESTest com.google.javascript.jscomp.StrictWarningsGuard_ESTest com.google.javascript.jscomp.SyntacticScopeCreator_ESTest com.google.javascript.jscomp.CheckProvides_ESTest com.google.javascript.jscomp.ErrorFormat_ESTest com.google.javascript.jscomp.TypedScopeCreator_ESTest com.google.javascript.jscomp.PassConfig_ESTest com.google.javascript.jscomp.CodingConventions_ESTest com.google.javascript.jscomp.JSModule_ESTest com.google.javascript.jscomp.NodeUtil_ESTest com.google.javascript.jscomp.JsAst_ESTest com.google.javascript.jscomp.DiagnosticGroup_ESTest com.google.javascript.jscomp.CheckGlobalNames_ESTest com.google.javascript.jscomp.CodeChangeHandler_ESTest com.google.javascript.jscomp.PrepareAst_ESTest com.google.javascript.jscomp.JsMessageVisitor_ESTest com.google.javascript.jscomp.CheckGlobalThis_ESTest com.google.javascript.jscomp.CreateSyntheticBlocks_ESTest com.google.javascript.jscomp.TypeValidator_ESTest com.google.javascript.jscomp.DefaultPassConfig_ESTest com.google.javascript.jscomp.JSModuleGraph_ESTest com.google.javascript.jscomp.CodeConsumer_ESTest com.google.javascript.jscomp.StrictModeCheck_ESTest com.google.javascript.jscomp.CrossModuleMethodMotion_ESTest com.google.javascript.jscomp.graph.DiGraph_ESTest com.google.javascript.jscomp.graph.LinkedDirectedGraph_ESTest com.google.javascript.jscomp.AstValidator_ESTest com.google.javascript.jscomp.Normalize_ESTest com.google.javascript.jscomp.TypeCheck_ESTest com.google.javascript.jscomp.LightweightMessageFormatter_ESTest com.google.javascript.jscomp.SourceFile_ESTest com.google.javascript.jscomp.PhaseOptimizer_ESTest com.google.javascript.jscomp.SyntheticAst_ESTest com.google.javascript.jscomp.CheckSideEffects_ESTest com.google.javascript.jscomp.DependencyOptions_ESTest com.google.javascript.jscomp.FunctionTypeBuilder_ESTest com.google.javascript.jscomp.OptimizeCalls_ESTest com.google.javascript.jscomp.SuppressDocWarningsGuard_ESTest com.google.javascript.jscomp.AbstractMessageFormatter_ESTest com.google.javascript.jscomp.Denormalize_ESTest com.google.javascript.jscomp.DiagnosticGroupWarningsGuard_ESTest com.google.javascript.jscomp.ClosureCodingConvention_ESTest com.google.javascript.jscomp.ProcessCommonJSModules_ESTest com.google.javascript.jscomp.CodeGenerator_ESTest com.google.javascript.jscomp.ControlStructureCheck_ESTest com.google.javascript.jscomp.NodeTraversal_ESTest com.google.javascript.jscomp.CheckLevel_ESTest com.google.javascript.jscomp.Compiler_ESTest com.google.javascript.jscomp.type.SemanticReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.type.ChainableReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.GoogleCodingConvention_ESTest com.google.javascript.jscomp.ComposeWarningsGuard_ESTest com.google.javascript.jscomp.ProcessDefines_ESTest com.google.javascript.jscomp.SourceMap_ESTest com.google.javascript.jscomp.Result_ESTest com.google.javascript.jscomp.AbstractCompiler_ESTest com.google.javascript.jscomp.CompilerOptions_ESTest com.google.javascript.jscomp.CleanupPasses_ESTest com.google.javascript.jscomp.DiagnosticType_ESTest com.google.javascript.jscomp.CompilerInput_ESTest com.google.javascript.jscomp.deps.JsFileLineParser_ESTest com.google.javascript.jscomp.deps.JsFileParser_ESTest com.google.javascript.jscomp.deps.SimpleDependencyInfo_ESTest com.google.javascript.jscomp.deps.SortedDependencies_ESTest com.google.javascript.jscomp.DiagnosticGroups_ESTest com.google.javascript.jscomp.WarningsGuard_ESTest com.google.javascript.jscomp.CheckRegExp_ESTest com.google.javascript.jscomp.ControlFlowGraph_ESTest com.google.javascript.jscomp.Scope_ESTest com.google.javascript.jscomp.GlobalNamespace_ESTest com.google.javascript.jscomp.parsing.Config_ESTest com.google.javascript.jscomp.parsing.IRFactory_ESTest com.google.javascript.jscomp.parsing.JsDocInfoParser_ESTest com.google.javascript.jscomp.parsing.ParserRunner_ESTest com.google.javascript.jscomp.parsing.JsDocTokenStream_ESTest com.google.javascript.jscomp.CodePrinter_ESTest com.google.javascript.jscomp.Tracer_ESTest com.google.javascript.jscomp.AnonymousFunctionNamingPolicy_ESTest com.google.javascript.jscomp.MakeDeclaredNamesUnique_ESTest com.google.javascript.jscomp.CheckUnreachableCode_ESTest com.google.javascript.jscomp.CheckDebuggerStatement_ESTest com.google.javascript.jscomp.BasicErrorManager_ESTest com.google.javascript.jscomp.ReplaceIdGenerators_ESTest com.google.javascript.jscomp.PassFactory_ESTest com.google.javascript.jscomp.LoggerErrorManager_ESTest com.google.javascript.jscomp.ProcessTweaks_ESTest com.google.javascript.jscomp.VarCheck_ESTest com.google.javascript.jscomp.CheckAccessControls_ESTest com.google.javascript.jscomp.ReferenceCollectingCallback_ESTest com.google.javascript.jscomp.RhinoErrorReporter_ESTest com.google.javascript.jscomp.JSError_ESTest com.google.javascript.jscomp.VariableReferenceCheck_ESTest com.google.javascript.rhino.ScriptRuntime_ESTest com.google.javascript.rhino.SourcePosition_ESTest com.google.javascript.rhino.IR_ESTest com.google.javascript.rhino.InputId_ESTest com.google.javascript.rhino.JSDocInfo_ESTest com.google.javascript.rhino.JSDocInfoBuilder_ESTest com.google.javascript.rhino.jstype.FunctionParamBuilder_ESTest com.google.javascript.rhino.jstype.StringType_ESTest com.google.javascript.rhino.jstype.ValueType_ESTest com.google.javascript.rhino.jstype.JSTypeRegistry_ESTest com.google.javascript.rhino.jstype.NoType_ESTest com.google.javascript.rhino.jstype.NoResolvedType_ESTest com.google.javascript.rhino.jstype.UnresolvedTypeExpression_ESTest com.google.javascript.rhino.jstype.ProxyObjectType_ESTest com.google.javascript.rhino.jstype.PrototypeObjectType_ESTest com.google.javascript.rhino.jstype.FunctionType_ESTest com.google.javascript.rhino.jstype.NullType_ESTest com.google.javascript.rhino.jstype.NoObjectType_ESTest com.google.javascript.rhino.jstype.FunctionBuilder_ESTest com.google.javascript.rhino.jstype.BooleanType_ESTest com.google.javascript.rhino.jstype.ParameterizedType_ESTest com.google.javascript.rhino.jstype.IndexedType_ESTest com.google.javascript.rhino.jstype.TemplateType_ESTest com.google.javascript.rhino.jstype.ErrorFunctionType_ESTest com.google.javascript.rhino.jstype.UnknownType_ESTest com.google.javascript.rhino.jstype.UnionTypeBuilder_ESTest com.google.javascript.rhino.jstype.JSType_ESTest com.google.javascript.rhino.jstype.NamedType_ESTest com.google.javascript.rhino.jstype.ObjectType_ESTest com.google.javascript.rhino.jstype.AllType_ESTest com.google.javascript.rhino.jstype.NumberType_ESTest com.google.javascript.rhino.jstype.ArrowType_ESTest com.google.javascript.rhino.jstype.UnionType_ESTest com.google.javascript.rhino.jstype.InstanceObjectType_ESTest com.google.javascript.rhino.jstype.VoidType_ESTest com.google.javascript.rhino.Node_ESTest com.google.javascript.rhino.JSTypeExpression_ESTest > resultJunity.txt
```

A lista de classes passadas para o Junit no comando acima foi obtida pelo comando find: 

``` bash
find outputTeste/Closure/evosuite-branch/1/ -name "*.java" > class.txt
```

e pela execução do seguinte script: **parserClasses.py**

Após executar obtivemos $50$ erros:

![alt text](pic/junit4_evo_testes_sobre_versao_bug.png)

## Rodando (MAC OS local) os testes unitários gerados no Lucy para a versão fixed

A compilação dos testes para a versão fixed usa o mesmo comando alterando apenas um parâmetro (o jar da versão fixed ao invés da versão bug dentro do classpath):


``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-branch/1/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-branch/1/ -name "*.java")
```

Uma classe não compila: **Graph_ESTest.java** segue abaixo o erro de execução encontrado (como na execução de cima):

![alt text](pic/erroCompFixed.png)

Removendo a classe com erro:

``` bash
--Removi a classe com erro
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-branch/1/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-branch/1/ -name "*.java" -and -not -name "Graph_ESTest.java")
``` 

foi possível compilar perfeitamente ($38$ warnings sobre uso de lib deprecated). Em seguida, executamos as classes de teste geradas pelo evosuite!

``` bash
java -cp .:"hamcrest-core-1.3.jar:junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-branch/1/" org.junit.runner.JUnitCore com.google.javascript.jscomp.DisambiguateProperties_ESTest com.google.javascript.jscomp.CodingConvention_ESTest com.google.javascript.jscomp.ConstCheck_ESTest com.google.javascript.jscomp.StrictWarningsGuard_ESTest com.google.javascript.jscomp.SyntacticScopeCreator_ESTest com.google.javascript.jscomp.CheckProvides_ESTest com.google.javascript.jscomp.ErrorFormat_ESTest com.google.javascript.jscomp.TypedScopeCreator_ESTest com.google.javascript.jscomp.PassConfig_ESTest com.google.javascript.jscomp.CodingConventions_ESTest com.google.javascript.jscomp.JSModule_ESTest com.google.javascript.jscomp.NodeUtil_ESTest com.google.javascript.jscomp.JsAst_ESTest com.google.javascript.jscomp.DiagnosticGroup_ESTest com.google.javascript.jscomp.CheckGlobalNames_ESTest com.google.javascript.jscomp.CodeChangeHandler_ESTest com.google.javascript.jscomp.PrepareAst_ESTest com.google.javascript.jscomp.JsMessageVisitor_ESTest com.google.javascript.jscomp.CheckGlobalThis_ESTest com.google.javascript.jscomp.CreateSyntheticBlocks_ESTest com.google.javascript.jscomp.TypeValidator_ESTest com.google.javascript.jscomp.DefaultPassConfig_ESTest com.google.javascript.jscomp.JSModuleGraph_ESTest com.google.javascript.jscomp.CodeConsumer_ESTest com.google.javascript.jscomp.StrictModeCheck_ESTest com.google.javascript.jscomp.CrossModuleMethodMotion_ESTest com.google.javascript.jscomp.graph.DiGraph_ESTest com.google.javascript.jscomp.graph.LinkedDirectedGraph_ESTest com.google.javascript.jscomp.AstValidator_ESTest com.google.javascript.jscomp.Normalize_ESTest com.google.javascript.jscomp.TypeCheck_ESTest com.google.javascript.jscomp.LightweightMessageFormatter_ESTest com.google.javascript.jscomp.SourceFile_ESTest com.google.javascript.jscomp.PhaseOptimizer_ESTest com.google.javascript.jscomp.SyntheticAst_ESTest com.google.javascript.jscomp.CheckSideEffects_ESTest com.google.javascript.jscomp.DependencyOptions_ESTest com.google.javascript.jscomp.FunctionTypeBuilder_ESTest com.google.javascript.jscomp.OptimizeCalls_ESTest com.google.javascript.jscomp.SuppressDocWarningsGuard_ESTest com.google.javascript.jscomp.AbstractMessageFormatter_ESTest com.google.javascript.jscomp.Denormalize_ESTest com.google.javascript.jscomp.DiagnosticGroupWarningsGuard_ESTest com.google.javascript.jscomp.ClosureCodingConvention_ESTest com.google.javascript.jscomp.ProcessCommonJSModules_ESTest com.google.javascript.jscomp.CodeGenerator_ESTest com.google.javascript.jscomp.ControlStructureCheck_ESTest com.google.javascript.jscomp.NodeTraversal_ESTest com.google.javascript.jscomp.CheckLevel_ESTest com.google.javascript.jscomp.Compiler_ESTest com.google.javascript.jscomp.type.SemanticReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.type.ChainableReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.GoogleCodingConvention_ESTest com.google.javascript.jscomp.ComposeWarningsGuard_ESTest com.google.javascript.jscomp.ProcessDefines_ESTest com.google.javascript.jscomp.SourceMap_ESTest com.google.javascript.jscomp.Result_ESTest com.google.javascript.jscomp.AbstractCompiler_ESTest com.google.javascript.jscomp.CompilerOptions_ESTest com.google.javascript.jscomp.CleanupPasses_ESTest com.google.javascript.jscomp.DiagnosticType_ESTest com.google.javascript.jscomp.CompilerInput_ESTest com.google.javascript.jscomp.deps.JsFileLineParser_ESTest com.google.javascript.jscomp.deps.JsFileParser_ESTest com.google.javascript.jscomp.deps.SimpleDependencyInfo_ESTest com.google.javascript.jscomp.deps.SortedDependencies_ESTest com.google.javascript.jscomp.DiagnosticGroups_ESTest com.google.javascript.jscomp.WarningsGuard_ESTest com.google.javascript.jscomp.CheckRegExp_ESTest com.google.javascript.jscomp.ControlFlowGraph_ESTest com.google.javascript.jscomp.Scope_ESTest com.google.javascript.jscomp.GlobalNamespace_ESTest com.google.javascript.jscomp.parsing.Config_ESTest com.google.javascript.jscomp.parsing.IRFactory_ESTest com.google.javascript.jscomp.parsing.JsDocInfoParser_ESTest com.google.javascript.jscomp.parsing.ParserRunner_ESTest com.google.javascript.jscomp.parsing.JsDocTokenStream_ESTest com.google.javascript.jscomp.CodePrinter_ESTest com.google.javascript.jscomp.Tracer_ESTest com.google.javascript.jscomp.AnonymousFunctionNamingPolicy_ESTest com.google.javascript.jscomp.MakeDeclaredNamesUnique_ESTest com.google.javascript.jscomp.CheckUnreachableCode_ESTest com.google.javascript.jscomp.CheckDebuggerStatement_ESTest com.google.javascript.jscomp.BasicErrorManager_ESTest com.google.javascript.jscomp.ReplaceIdGenerators_ESTest com.google.javascript.jscomp.PassFactory_ESTest com.google.javascript.jscomp.LoggerErrorManager_ESTest com.google.javascript.jscomp.ProcessTweaks_ESTest com.google.javascript.jscomp.VarCheck_ESTest com.google.javascript.jscomp.CheckAccessControls_ESTest com.google.javascript.jscomp.ReferenceCollectingCallback_ESTest com.google.javascript.jscomp.RhinoErrorReporter_ESTest com.google.javascript.jscomp.JSError_ESTest com.google.javascript.jscomp.VariableReferenceCheck_ESTest com.google.javascript.rhino.ScriptRuntime_ESTest com.google.javascript.rhino.SourcePosition_ESTest com.google.javascript.rhino.IR_ESTest com.google.javascript.rhino.InputId_ESTest com.google.javascript.rhino.JSDocInfo_ESTest com.google.javascript.rhino.JSDocInfoBuilder_ESTest com.google.javascript.rhino.jstype.FunctionParamBuilder_ESTest com.google.javascript.rhino.jstype.StringType_ESTest com.google.javascript.rhino.jstype.ValueType_ESTest com.google.javascript.rhino.jstype.JSTypeRegistry_ESTest com.google.javascript.rhino.jstype.NoType_ESTest com.google.javascript.rhino.jstype.NoResolvedType_ESTest com.google.javascript.rhino.jstype.UnresolvedTypeExpression_ESTest com.google.javascript.rhino.jstype.ProxyObjectType_ESTest com.google.javascript.rhino.jstype.PrototypeObjectType_ESTest com.google.javascript.rhino.jstype.FunctionType_ESTest com.google.javascript.rhino.jstype.NullType_ESTest com.google.javascript.rhino.jstype.NoObjectType_ESTest com.google.javascript.rhino.jstype.FunctionBuilder_ESTest com.google.javascript.rhino.jstype.BooleanType_ESTest com.google.javascript.rhino.jstype.ParameterizedType_ESTest com.google.javascript.rhino.jstype.IndexedType_ESTest com.google.javascript.rhino.jstype.TemplateType_ESTest com.google.javascript.rhino.jstype.ErrorFunctionType_ESTest com.google.javascript.rhino.jstype.UnknownType_ESTest com.google.javascript.rhino.jstype.UnionTypeBuilder_ESTest com.google.javascript.rhino.jstype.JSType_ESTest com.google.javascript.rhino.jstype.NamedType_ESTest com.google.javascript.rhino.jstype.ObjectType_ESTest com.google.javascript.rhino.jstype.AllType_ESTest com.google.javascript.rhino.jstype.NumberType_ESTest com.google.javascript.rhino.jstype.ArrowType_ESTest com.google.javascript.rhino.jstype.UnionType_ESTest com.google.javascript.rhino.jstype.InstanceObjectType_ESTest com.google.javascript.rhino.jstype.VoidType_ESTest com.google.javascript.rhino.Node_ESTest com.google.javascript.rhino.JSTypeExpression_ESTest > resultJunity.txt
```

Após executar obtivemos $51$ erros:

![alt text](pic/junit4_evo_testes_sobre_versao_fixed.png)

# Observações: execução weakmutation

## Rodando (MAC OS local) os testes unitários gerados no Lucy para a versão bug

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-weakmutation/2/Closure-18b-evosuite-weakmutation.2/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-weakmutation/2/ -name "*.java")
```

Uma classe não compila: **Graph_ESTest.java** segue abaixo o erro de execução encontrado (como na execução de cima):

![alt text](pic/erroCompBug_weak.png)

Removendo a classe com erro:

``` bash
--Removi a classe com erro
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-weakmutation/2/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-weakmutation/2/ -name "*.java" -and -not -name "Graph_ESTest.java")
``` 

foi possível compilar perfeitamente ($33$ warnings sobre uso de lib deprecated). Em seguida, executamos as classes de teste geradas pelo evosuite!

``` bash
java -cp .:"hamcrest-core-1.3.jar:junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-weakmutation/2/Closure-18b-evosuite-weakmutation.2/" org.junit.runner.JUnitCore google.javascript.jscomp.DisambiguateProperties_ESTest google.javascript.jscomp.CodingConvention_ESTest google.javascript.jscomp.ConstCheck_ESTest google.javascript.jscomp.StrictWarningsGuard_ESTest google.javascript.jscomp.SyntacticScopeCreator_ESTest google.javascript.jscomp.CheckProvides_ESTest google.javascript.jscomp.ErrorFormat_ESTest google.javascript.jscomp.TypedScopeCreator_ESTest google.javascript.jscomp.PassConfig_ESTest google.javascript.jscomp.CodingConventions_ESTest google.javascript.jscomp.JSModule_ESTest google.javascript.jscomp.NodeUtil_ESTest google.javascript.jscomp.JsAst_ESTest google.javascript.jscomp.DiagnosticGroup_ESTest google.javascript.jscomp.CheckGlobalNames_ESTest google.javascript.jscomp.CodeChangeHandler_ESTest google.javascript.jscomp.PrepareAst_ESTest google.javascript.jscomp.JsMessageVisitor_ESTest google.javascript.jscomp.CheckGlobalThis_ESTest google.javascript.jscomp.CreateSyntheticBlocks_ESTest google.javascript.jscomp.TypeValidator_ESTest google.javascript.jscomp.DefaultPassConfig_ESTest google.javascript.jscomp.JSModuleGraph_ESTest google.javascript.jscomp.CodeConsumer_ESTest google.javascript.jscomp.StrictModeCheck_ESTest google.javascript.jscomp.CrossModuleMethodMotion_ESTest google.javascript.jscomp.graph.DiGraph_ESTest google.javascript.jscomp.graph.LinkedDirectedGraph_ESTest google.javascript.jscomp.AstValidator_ESTest google.javascript.jscomp.Normalize_ESTest google.javascript.jscomp.TypeCheck_ESTest google.javascript.jscomp.LightweightMessageFormatter_ESTest google.javascript.jscomp.SourceFile_ESTest google.javascript.jscomp.PhaseOptimizer_ESTest google.javascript.jscomp.SyntheticAst_ESTest google.javascript.jscomp.CheckSideEffects_ESTest google.javascript.jscomp.DependencyOptions_ESTest google.javascript.jscomp.FunctionTypeBuilder_ESTest google.javascript.jscomp.OptimizeCalls_ESTest google.javascript.jscomp.AbstractMessageFormatter_ESTest google.javascript.jscomp.Denormalize_ESTest google.javascript.jscomp.DiagnosticGroupWarningsGuard_ESTest google.javascript.jscomp.ClosureCodingConvention_ESTest google.javascript.jscomp.SourceExcerptProvider_ESTest google.javascript.jscomp.ProcessCommonJSModules_ESTest google.javascript.jscomp.CodeGenerator_ESTest google.javascript.jscomp.ControlStructureCheck_ESTest google.javascript.jscomp.NodeTraversal_ESTest google.javascript.jscomp.CheckLevel_ESTest google.javascript.jscomp.Compiler_ESTest google.javascript.jscomp.type.SemanticReverseAbstractInterpreter_ESTest google.javascript.jscomp.type.ChainableReverseAbstractInterpreter_ESTest google.javascript.jscomp.GoogleCodingConvention_ESTest google.javascript.jscomp.ComposeWarningsGuard_ESTest google.javascript.jscomp.ProcessDefines_ESTest google.javascript.jscomp.SourceMap_ESTest google.javascript.jscomp.Result_ESTest google.javascript.jscomp.AbstractCompiler_ESTest google.javascript.jscomp.CompilerOptions_ESTest google.javascript.jscomp.CleanupPasses_ESTest google.javascript.jscomp.DiagnosticType_ESTest google.javascript.jscomp.CompilerInput_ESTest google.javascript.jscomp.deps.JsFileLineParser_ESTest google.javascript.jscomp.deps.JsFileParser_ESTest google.javascript.jscomp.deps.SimpleDependencyInfo_ESTest google.javascript.jscomp.deps.SortedDependencies_ESTest google.javascript.jscomp.DiagnosticGroups_ESTest google.javascript.jscomp.WarningsGuard_ESTest google.javascript.jscomp.CheckRegExp_ESTest google.javascript.jscomp.ControlFlowGraph_ESTest google.javascript.jscomp.Scope_ESTest google.javascript.jscomp.GlobalNamespace_ESTest google.javascript.jscomp.parsing.Config_ESTest google.javascript.jscomp.parsing.IRFactory_ESTest google.javascript.jscomp.parsing.JsDocInfoParser_ESTest google.javascript.jscomp.parsing.ParserRunner_ESTest google.javascript.jscomp.parsing.JsDocTokenStream_ESTest google.javascript.jscomp.CodePrinter_ESTest google.javascript.jscomp.Tracer_ESTest google.javascript.jscomp.MakeDeclaredNamesUnique_ESTest google.javascript.jscomp.CheckDebuggerStatement_ESTest google.javascript.jscomp.BasicErrorManager_ESTest google.javascript.jscomp.ReplaceIdGenerators_ESTest google.javascript.jscomp.PassFactory_ESTest google.javascript.jscomp.LoggerErrorManager_ESTest google.javascript.jscomp.ProcessTweaks_ESTest google.javascript.jscomp.VarCheck_ESTest google.javascript.jscomp.CheckAccessControls_ESTest google.javascript.jscomp.ReferenceCollectingCallback_ESTest google.javascript.jscomp.RhinoErrorReporter_ESTest google.javascript.jscomp.JSError_ESTest google.javascript.jscomp.VariableReferenceCheck_ESTest google.javascript.rhino.ScriptRuntime_ESTest google.javascript.rhino.SourcePosition_ESTest google.javascript.rhino.IR_ESTest google.javascript.rhino.InputId_ESTest google.javascript.rhino.JSDocInfo_ESTest google.javascript.rhino.JSDocInfoBuilder_ESTest google.javascript.rhino.jstype.FunctionParamBuilder_ESTest google.javascript.rhino.jstype.StringType_ESTest google.javascript.rhino.jstype.ValueType_ESTest google.javascript.rhino.jstype.JSTypeRegistry_ESTest google.javascript.rhino.jstype.NoType_ESTest google.javascript.rhino.jstype.NoResolvedType_ESTest google.javascript.rhino.jstype.UnresolvedTypeExpression_ESTest google.javascript.rhino.jstype.ProxyObjectType_ESTest google.javascript.rhino.jstype.PrototypeObjectType_ESTest google.javascript.rhino.jstype.FunctionType_ESTest google.javascript.rhino.jstype.NullType_ESTest google.javascript.rhino.jstype.NoObjectType_ESTest google.javascript.rhino.jstype.FunctionBuilder_ESTest google.javascript.rhino.jstype.BooleanType_ESTest google.javascript.rhino.jstype.ParameterizedType_ESTest google.javascript.rhino.jstype.IndexedType_ESTest google.javascript.rhino.jstype.TemplateType_ESTest google.javascript.rhino.jstype.UnknownType_ESTest google.javascript.rhino.jstype.UnionTypeBuilder_ESTest google.javascript.rhino.jstype.JSType_ESTest google.javascript.rhino.jstype.NamedType_ESTest google.javascript.rhino.jstype.ObjectType_ESTest google.javascript.rhino.jstype.AllType_ESTest google.javascript.rhino.jstype.NumberType_ESTest google.javascript.rhino.jstype.ArrowType_ESTest google.javascript.rhino.jstype.UnionType_ESTest google.javascript.rhino.jstype.InstanceObjectType_ESTest google.javascript.rhino.jstype.VoidType_ESTest google.javascript.rhino.JSTypeExpression_ESTest > resultJunityWeak.txt
```

``` bash
find outputTeste/Closure/evosuite-weakmutation/2/Closure-18b-evosuite-weakmutation.2 -name "*.class" > classweak.txt
```

Após executar obtivemos 34 erros:

![alt text](pic/junity4_evo_testes_sobre_versao_bug_weakmutation.png)

## Rodando (MAC OS local) os testes unitários gerados no Lucy para a versão fixed

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-weakmutation/2/Closure-18b-evosuite-weakmutation.2/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-weakmutation/2/ -name "*.java")
```

Uma classe não compila: **Graph_ESTest.java** segue abaixo o erro de execução encontrado (como na execução de cima):

![alt text](pic/erroCompFixed_weak.png)

``` bash
--Removi a classe com erro
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-weakmutation/2/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-weakmutation/2/ -name "*.java" -and -not -name "Graph_ESTest.java")
``` 

foi possível compilar perfeitamente ($33$ warnings sobre uso de lib deprecated). Em seguida, executamos as classes de teste geradas pelo evosuite!

``` bash
java -cp .:"hamcrest-core-1.3.jar:junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-weakmutation/2/Closure-18b-evosuite-weakmutation.2/" org.junit.runner.JUnitCore google.javascript.jscomp.DisambiguateProperties_ESTest google.javascript.jscomp.CodingConvention_ESTest google.javascript.jscomp.ConstCheck_ESTest google.javascript.jscomp.StrictWarningsGuard_ESTest google.javascript.jscomp.SyntacticScopeCreator_ESTest google.javascript.jscomp.CheckProvides_ESTest google.javascript.jscomp.ErrorFormat_ESTest google.javascript.jscomp.TypedScopeCreator_ESTest google.javascript.jscomp.PassConfig_ESTest google.javascript.jscomp.CodingConventions_ESTest google.javascript.jscomp.JSModule_ESTest google.javascript.jscomp.NodeUtil_ESTest google.javascript.jscomp.JsAst_ESTest google.javascript.jscomp.DiagnosticGroup_ESTest google.javascript.jscomp.CheckGlobalNames_ESTest google.javascript.jscomp.CodeChangeHandler_ESTest google.javascript.jscomp.PrepareAst_ESTest google.javascript.jscomp.JsMessageVisitor_ESTest google.javascript.jscomp.CheckGlobalThis_ESTest google.javascript.jscomp.CreateSyntheticBlocks_ESTest google.javascript.jscomp.TypeValidator_ESTest google.javascript.jscomp.DefaultPassConfig_ESTest google.javascript.jscomp.JSModuleGraph_ESTest google.javascript.jscomp.CodeConsumer_ESTest google.javascript.jscomp.StrictModeCheck_ESTest google.javascript.jscomp.CrossModuleMethodMotion_ESTest google.javascript.jscomp.graph.DiGraph_ESTest google.javascript.jscomp.graph.LinkedDirectedGraph_ESTest google.javascript.jscomp.AstValidator_ESTest google.javascript.jscomp.Normalize_ESTest google.javascript.jscomp.TypeCheck_ESTest google.javascript.jscomp.LightweightMessageFormatter_ESTest google.javascript.jscomp.SourceFile_ESTest google.javascript.jscomp.PhaseOptimizer_ESTest google.javascript.jscomp.SyntheticAst_ESTest google.javascript.jscomp.CheckSideEffects_ESTest google.javascript.jscomp.DependencyOptions_ESTest google.javascript.jscomp.FunctionTypeBuilder_ESTest google.javascript.jscomp.OptimizeCalls_ESTest google.javascript.jscomp.AbstractMessageFormatter_ESTest google.javascript.jscomp.Denormalize_ESTest google.javascript.jscomp.DiagnosticGroupWarningsGuard_ESTest google.javascript.jscomp.ClosureCodingConvention_ESTest google.javascript.jscomp.SourceExcerptProvider_ESTest google.javascript.jscomp.ProcessCommonJSModules_ESTest google.javascript.jscomp.CodeGenerator_ESTest google.javascript.jscomp.ControlStructureCheck_ESTest google.javascript.jscomp.NodeTraversal_ESTest google.javascript.jscomp.CheckLevel_ESTest google.javascript.jscomp.Compiler_ESTest google.javascript.jscomp.type.SemanticReverseAbstractInterpreter_ESTest google.javascript.jscomp.type.ChainableReverseAbstractInterpreter_ESTest google.javascript.jscomp.GoogleCodingConvention_ESTest google.javascript.jscomp.ComposeWarningsGuard_ESTest google.javascript.jscomp.ProcessDefines_ESTest google.javascript.jscomp.SourceMap_ESTest google.javascript.jscomp.Result_ESTest google.javascript.jscomp.AbstractCompiler_ESTest google.javascript.jscomp.CompilerOptions_ESTest google.javascript.jscomp.CleanupPasses_ESTest google.javascript.jscomp.DiagnosticType_ESTest google.javascript.jscomp.CompilerInput_ESTest google.javascript.jscomp.deps.JsFileLineParser_ESTest google.javascript.jscomp.deps.JsFileParser_ESTest google.javascript.jscomp.deps.SimpleDependencyInfo_ESTest google.javascript.jscomp.deps.SortedDependencies_ESTest google.javascript.jscomp.DiagnosticGroups_ESTest google.javascript.jscomp.WarningsGuard_ESTest google.javascript.jscomp.CheckRegExp_ESTest google.javascript.jscomp.ControlFlowGraph_ESTest google.javascript.jscomp.Scope_ESTest google.javascript.jscomp.GlobalNamespace_ESTest google.javascript.jscomp.parsing.Config_ESTest google.javascript.jscomp.parsing.IRFactory_ESTest google.javascript.jscomp.parsing.JsDocInfoParser_ESTest google.javascript.jscomp.parsing.ParserRunner_ESTest google.javascript.jscomp.parsing.JsDocTokenStream_ESTest google.javascript.jscomp.CodePrinter_ESTest google.javascript.jscomp.Tracer_ESTest google.javascript.jscomp.MakeDeclaredNamesUnique_ESTest google.javascript.jscomp.CheckDebuggerStatement_ESTest google.javascript.jscomp.BasicErrorManager_ESTest google.javascript.jscomp.ReplaceIdGenerators_ESTest google.javascript.jscomp.PassFactory_ESTest google.javascript.jscomp.LoggerErrorManager_ESTest google.javascript.jscomp.ProcessTweaks_ESTest google.javascript.jscomp.VarCheck_ESTest google.javascript.jscomp.CheckAccessControls_ESTest google.javascript.jscomp.ReferenceCollectingCallback_ESTest google.javascript.jscomp.RhinoErrorReporter_ESTest google.javascript.jscomp.JSError_ESTest google.javascript.jscomp.VariableReferenceCheck_ESTest google.javascript.rhino.ScriptRuntime_ESTest google.javascript.rhino.SourcePosition_ESTest google.javascript.rhino.IR_ESTest google.javascript.rhino.InputId_ESTest google.javascript.rhino.JSDocInfo_ESTest google.javascript.rhino.JSDocInfoBuilder_ESTest google.javascript.rhino.jstype.FunctionParamBuilder_ESTest google.javascript.rhino.jstype.StringType_ESTest google.javascript.rhino.jstype.ValueType_ESTest google.javascript.rhino.jstype.JSTypeRegistry_ESTest google.javascript.rhino.jstype.NoType_ESTest google.javascript.rhino.jstype.NoResolvedType_ESTest google.javascript.rhino.jstype.UnresolvedTypeExpression_ESTest google.javascript.rhino.jstype.ProxyObjectType_ESTest google.javascript.rhino.jstype.PrototypeObjectType_ESTest google.javascript.rhino.jstype.FunctionType_ESTest google.javascript.rhino.jstype.NullType_ESTest google.javascript.rhino.jstype.NoObjectType_ESTest google.javascript.rhino.jstype.FunctionBuilder_ESTest google.javascript.rhino.jstype.BooleanType_ESTest google.javascript.rhino.jstype.ParameterizedType_ESTest google.javascript.rhino.jstype.IndexedType_ESTest google.javascript.rhino.jstype.TemplateType_ESTest google.javascript.rhino.jstype.UnknownType_ESTest google.javascript.rhino.jstype.UnionTypeBuilder_ESTest google.javascript.rhino.jstype.JSType_ESTest google.javascript.rhino.jstype.NamedType_ESTest google.javascript.rhino.jstype.ObjectType_ESTest google.javascript.rhino.jstype.AllType_ESTest google.javascript.rhino.jstype.NumberType_ESTest google.javascript.rhino.jstype.ArrowType_ESTest google.javascript.rhino.jstype.UnionType_ESTest google.javascript.rhino.jstype.InstanceObjectType_ESTest google.javascript.rhino.jstype.VoidType_ESTest google.javascript.rhino.JSTypeExpression_ESTest > resultJunityWeak.txt
```

``` bash
find outputTeste/Closure/evosuite-weakmutation/2/Closure-18b-evosuite-weakmutation.2 -name "*.class" > classweak.txt
```

![alt text](pic/junit4_evo_testes_sobre_versao_fixed_weak.png)

# Observações: execução strongmutation

## Rodando (MAC OS local) os testes unitários gerados no Lucy para a versão bug

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/ -name "*.java")
```

Uma classe não compila: **Graph_ESTest.java** segue abaixo o erro de execução encontrado (como na execução de cima):

![alt text](pic/erroCompBug_strong.png)

Removendo a classe com erro:

``` bash
--Removi a classe com erro
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/ -name "*.java" -and -not -name "Graph_ESTest.java")
``` 

foi possível compilar perfeitamente ($100$ warnings sobre uso de lib deprecated). Em seguida, executamos as classes de teste geradas pelo evosuite!

``` bash
java -cp .:"hamcrest-core-1.3.jar:junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_bug/build/compiler.jar:./outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/" org.junit.runner.JUnitCore com.google.javascript.jscomp.CodingConvention_ESTest com.google.javascript.jscomp.ConstCheck_ESTest com.google.javascript.jscomp.StrictWarningsGuard_ESTest com.google.javascript.jscomp.SyntacticScopeCreator_ESTest com.google.javascript.jscomp.CheckProvides_ESTest com.google.javascript.jscomp.TypedScopeCreator_ESTest com.google.javascript.jscomp.PassConfig_ESTest com.google.javascript.jscomp.CodingConventions_ESTest com.google.javascript.jscomp.JSModule_ESTest com.google.javascript.jscomp.JsAst_ESTest com.google.javascript.jscomp.DiagnosticGroup_ESTest com.google.javascript.jscomp.CheckGlobalNames_ESTest com.google.javascript.jscomp.CodeChangeHandler_ESTest com.google.javascript.jscomp.PrepareAst_ESTest com.google.javascript.jscomp.JsMessageVisitor_ESTest com.google.javascript.jscomp.TypeValidator_ESTest com.google.javascript.jscomp.JSModuleGraph_ESTest com.google.javascript.jscomp.CodeConsumer_ESTest com.google.javascript.jscomp.CrossModuleMethodMotion_ESTest com.google.javascript.jscomp.graph.DiGraph_ESTest com.google.javascript.jscomp.graph.LinkedDirectedGraph_ESTest com.google.javascript.jscomp.AstValidator_ESTest com.google.javascript.jscomp.Normalize_ESTest com.google.javascript.jscomp.LightweightMessageFormatter_ESTest com.google.javascript.jscomp.SourceFile_ESTest com.google.javascript.jscomp.PhaseOptimizer_ESTest com.google.javascript.jscomp.CheckSideEffects_ESTest com.google.javascript.jscomp.DependencyOptions_ESTest com.google.javascript.jscomp.FunctionTypeBuilder_ESTest com.google.javascript.jscomp.OptimizeCalls_ESTest com.google.javascript.jscomp.SuppressDocWarningsGuard_ESTest com.google.javascript.jscomp.AbstractMessageFormatter_ESTest com.google.javascript.jscomp.Denormalize_ESTest com.google.javascript.jscomp.DiagnosticGroupWarningsGuard_ESTest com.google.javascript.jscomp.ClosureCodingConvention_ESTest com.google.javascript.jscomp.ProcessCommonJSModules_ESTest com.google.javascript.jscomp.CodeGenerator_ESTest com.google.javascript.jscomp.NodeTraversal_ESTest com.google.javascript.jscomp.CheckLevel_ESTest com.google.javascript.jscomp.type.SemanticReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.type.ChainableReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.GoogleCodingConvention_ESTest com.google.javascript.jscomp.ComposeWarningsGuard_ESTest com.google.javascript.jscomp.SourceMap_ESTest com.google.javascript.jscomp.AbstractCompiler_ESTest com.google.javascript.jscomp.DiagnosticType_ESTest com.google.javascript.jscomp.CompilerInput_ESTest com.google.javascript.jscomp.deps.JsFileLineParser_ESTest com.google.javascript.jscomp.deps.JsFileParser_ESTest com.google.javascript.jscomp.deps.SimpleDependencyInfo_ESTest com.google.javascript.jscomp.deps.SortedDependencies_ESTest com.google.javascript.jscomp.DiagnosticGroups_ESTest com.google.javascript.jscomp.WarningsGuard_ESTest com.google.javascript.jscomp.CheckRegExp_ESTest com.google.javascript.jscomp.ControlFlowGraph_ESTest com.google.javascript.jscomp.Scope_ESTest com.google.javascript.jscomp.GlobalNamespace_ESTest com.google.javascript.jscomp.parsing.Config_ESTest com.google.javascript.jscomp.parsing.IRFactory_ESTest com.google.javascript.jscomp.parsing.JsDocInfoParser_ESTest com.google.javascript.jscomp.parsing.ParserRunner_ESTest com.google.javascript.jscomp.parsing.JsDocTokenStream_ESTest com.google.javascript.jscomp.CodePrinter_ESTest com.google.javascript.jscomp.MakeDeclaredNamesUnique_ESTest com.google.javascript.jscomp.CheckDebuggerStatement_ESTest com.google.javascript.jscomp.BasicErrorManager_ESTest com.google.javascript.jscomp.ReplaceIdGenerators_ESTest com.google.javascript.jscomp.PassFactory_ESTest com.google.javascript.jscomp.ProcessTweaks_ESTest com.google.javascript.jscomp.ReferenceCollectingCallback_ESTest com.google.javascript.jscomp.JSError_ESTest com.google.javascript.rhino.ScriptRuntime_ESTest com.google.javascript.rhino.SourcePosition_ESTest com.google.javascript.rhino.IR_ESTest com.google.javascript.rhino.InputId_ESTest com.google.javascript.rhino.JSDocInfoBuilder_ESTest com.google.javascript.rhino.jstype.FunctionParamBuilder_ESTest com.google.javascript.rhino.jstype.StringType_ESTest com.google.javascript.rhino.jstype.ValueType_ESTest com.google.javascript.rhino.jstype.NoType_ESTest com.google.javascript.rhino.jstype.NoResolvedType_ESTest com.google.javascript.rhino.jstype.ProxyObjectType_ESTest com.google.javascript.rhino.jstype.NullType_ESTest com.google.javascript.rhino.jstype.NoObjectType_ESTest com.google.javascript.rhino.jstype.FunctionBuilder_ESTest com.google.javascript.rhino.jstype.BooleanType_ESTest com.google.javascript.rhino.jstype.ParameterizedType_ESTest com.google.javascript.rhino.jstype.IndexedType_ESTest com.google.javascript.rhino.jstype.TemplateType_ESTest com.google.javascript.rhino.jstype.UnknownType_ESTest com.google.javascript.rhino.jstype.UnionTypeBuilder_ESTest com.google.javascript.rhino.jstype.NamedType_ESTest com.google.javascript.rhino.jstype.ObjectType_ESTest com.google.javascript.rhino.jstype.AllType_ESTest com.google.javascript.rhino.jstype.NumberType_ESTest com.google.javascript.rhino.jstype.ArrowType_ESTest com.google.javascript.rhino.jstype.UnionType_ESTest com.google.javascript.rhino.jstype.InstanceObjectType_ESTest com.google.javascript.rhino.jstype.VoidType_ESTest com.google.javascript.rhino.JSTypeExpression_ESTest > resultJunityStrong.txt
```

``` bash
find outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3 -name "*.class" > classstrong.txt
```

![alt text](pic/junity4_evo_testes_sobre_versao_bug_strongmutation.png) 

## Rodando (MAC OS local) os testes unitários gerados no Lucy para a versão fixed

``` bash
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/ -name "*.java")
```

Uma classe não compila: **Graph_ESTest.java** segue abaixo o erro de execução encontrado (como na execução de cima):

![alt text](pic/erroCompFix_strong.png)

Removendo a classe com erro:

``` bash
--Removi a classe com erro
javac -cp .:"junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/com/google/javascript/jscomp/" $(find outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/ -name "*.java" -and -not -name "Graph_ESTest.java")
``` 

foi possível compilar perfeitamente ($100$ warnings sobre uso de lib deprecated). Em seguida, executamos as classes de teste geradas pelo evosuite!

java -cp .:"hamcrest-core-1.3.jar:junit-4.12.jar:evosuite-1.0.6.jar:./Clojure_18_fixed/build/compiler.jar:./outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3/" org.junit.runner.JUnitCore com.google.javascript.jscomp.CodingConvention_ESTest com.google.javascript.jscomp.ConstCheck_ESTest com.google.javascript.jscomp.StrictWarningsGuard_ESTest com.google.javascript.jscomp.SyntacticScopeCreator_ESTest com.google.javascript.jscomp.CheckProvides_ESTest com.google.javascript.jscomp.TypedScopeCreator_ESTest com.google.javascript.jscomp.PassConfig_ESTest com.google.javascript.jscomp.CodingConventions_ESTest com.google.javascript.jscomp.JSModule_ESTest com.google.javascript.jscomp.JsAst_ESTest com.google.javascript.jscomp.DiagnosticGroup_ESTest com.google.javascript.jscomp.CheckGlobalNames_ESTest com.google.javascript.jscomp.CodeChangeHandler_ESTest com.google.javascript.jscomp.PrepareAst_ESTest com.google.javascript.jscomp.JsMessageVisitor_ESTest com.google.javascript.jscomp.TypeValidator_ESTest com.google.javascript.jscomp.JSModuleGraph_ESTest com.google.javascript.jscomp.CodeConsumer_ESTest com.google.javascript.jscomp.CrossModuleMethodMotion_ESTest com.google.javascript.jscomp.graph.DiGraph_ESTest com.google.javascript.jscomp.graph.LinkedDirectedGraph_ESTest com.google.javascript.jscomp.AstValidator_ESTest com.google.javascript.jscomp.Normalize_ESTest com.google.javascript.jscomp.LightweightMessageFormatter_ESTest com.google.javascript.jscomp.SourceFile_ESTest com.google.javascript.jscomp.PhaseOptimizer_ESTest com.google.javascript.jscomp.CheckSideEffects_ESTest com.google.javascript.jscomp.DependencyOptions_ESTest com.google.javascript.jscomp.FunctionTypeBuilder_ESTest com.google.javascript.jscomp.OptimizeCalls_ESTest com.google.javascript.jscomp.SuppressDocWarningsGuard_ESTest com.google.javascript.jscomp.AbstractMessageFormatter_ESTest com.google.javascript.jscomp.Denormalize_ESTest com.google.javascript.jscomp.DiagnosticGroupWarningsGuard_ESTest com.google.javascript.jscomp.ClosureCodingConvention_ESTest com.google.javascript.jscomp.ProcessCommonJSModules_ESTest com.google.javascript.jscomp.CodeGenerator_ESTest com.google.javascript.jscomp.NodeTraversal_ESTest com.google.javascript.jscomp.CheckLevel_ESTest com.google.javascript.jscomp.type.SemanticReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.type.ChainableReverseAbstractInterpreter_ESTest com.google.javascript.jscomp.GoogleCodingConvention_ESTest com.google.javascript.jscomp.ComposeWarningsGuard_ESTest com.google.javascript.jscomp.SourceMap_ESTest com.google.javascript.jscomp.AbstractCompiler_ESTest com.google.javascript.jscomp.DiagnosticType_ESTest com.google.javascript.jscomp.CompilerInput_ESTest com.google.javascript.jscomp.deps.JsFileLineParser_ESTest com.google.javascript.jscomp.deps.JsFileParser_ESTest com.google.javascript.jscomp.deps.SimpleDependencyInfo_ESTest com.google.javascript.jscomp.deps.SortedDependencies_ESTest com.google.javascript.jscomp.DiagnosticGroups_ESTest com.google.javascript.jscomp.WarningsGuard_ESTest com.google.javascript.jscomp.CheckRegExp_ESTest com.google.javascript.jscomp.ControlFlowGraph_ESTest com.google.javascript.jscomp.Scope_ESTest com.google.javascript.jscomp.GlobalNamespace_ESTest com.google.javascript.jscomp.parsing.Config_ESTest com.google.javascript.jscomp.parsing.IRFactory_ESTest com.google.javascript.jscomp.parsing.JsDocInfoParser_ESTest com.google.javascript.jscomp.parsing.ParserRunner_ESTest com.google.javascript.jscomp.parsing.JsDocTokenStream_ESTest com.google.javascript.jscomp.CodePrinter_ESTest com.google.javascript.jscomp.MakeDeclaredNamesUnique_ESTest com.google.javascript.jscomp.CheckDebuggerStatement_ESTest com.google.javascript.jscomp.BasicErrorManager_ESTest com.google.javascript.jscomp.ReplaceIdGenerators_ESTest com.google.javascript.jscomp.PassFactory_ESTest com.google.javascript.jscomp.ProcessTweaks_ESTest com.google.javascript.jscomp.ReferenceCollectingCallback_ESTest com.google.javascript.jscomp.JSError_ESTest com.google.javascript.rhino.ScriptRuntime_ESTest com.google.javascript.rhino.SourcePosition_ESTest com.google.javascript.rhino.IR_ESTest com.google.javascript.rhino.InputId_ESTest com.google.javascript.rhino.JSDocInfoBuilder_ESTest com.google.javascript.rhino.jstype.FunctionParamBuilder_ESTest com.google.javascript.rhino.jstype.StringType_ESTest com.google.javascript.rhino.jstype.ValueType_ESTest com.google.javascript.rhino.jstype.NoType_ESTest com.google.javascript.rhino.jstype.NoResolvedType_ESTest com.google.javascript.rhino.jstype.ProxyObjectType_ESTest com.google.javascript.rhino.jstype.NullType_ESTest com.google.javascript.rhino.jstype.NoObjectType_ESTest com.google.javascript.rhino.jstype.FunctionBuilder_ESTest com.google.javascript.rhino.jstype.BooleanType_ESTest com.google.javascript.rhino.jstype.ParameterizedType_ESTest com.google.javascript.rhino.jstype.IndexedType_ESTest com.google.javascript.rhino.jstype.TemplateType_ESTest com.google.javascript.rhino.jstype.UnknownType_ESTest com.google.javascript.rhino.jstype.UnionTypeBuilder_ESTest com.google.javascript.rhino.jstype.NamedType_ESTest com.google.javascript.rhino.jstype.ObjectType_ESTest com.google.javascript.rhino.jstype.AllType_ESTest com.google.javascript.rhino.jstype.NumberType_ESTest com.google.javascript.rhino.jstype.ArrowType_ESTest com.google.javascript.rhino.jstype.UnionType_ESTest com.google.javascript.rhino.jstype.InstanceObjectType_ESTest com.google.javascript.rhino.jstype.VoidType_ESTest com.google.javascript.rhino.JSTypeExpression_ESTest > resultJunityStrong.txt

``` bash
find outputTeste/Closure/evosuite-strongmutation/3/Closure-18b-evosuite-strongmutation.3 -name "*.class" > classstrong.txt
```

![alt text](pic/junit4_evo_testes_sobre_versao_fixed_strong.png)

# Conclusões

Os testes gerados pelo evosuite não falharam mais ou menos nas versões bug x fixed. Na verdade foram obtidos os mesmos números de falhas.

# Next steps

1. Analisar os 101 erros de ambas as execuções, comparar as duas (intersecção)
2. Avaliar qual é o erro extra encontrado
3. Avaliar se o erro faz sentido, ou não
4. Testar o Junit 5